In [1]:
# reload modules
%load_ext autoreload
%autoreload 2

In [2]:
import os

# os.chdir("") # set the root directory of the project
os.chdir("/mnt/ssd3/ericzhou/sra-artifact/rq2") 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sra.dataloader import (
    get_covs_runs,
    get_run_paths,
    get_trial_paths,
    get_covs_trial,
)
from sra.cfggen import parse_ll, parse_dotfiles, gen_cfg_inter
from sra.mapper import get_bbs_fuzz, get_mapping, fuzzdat_to_obs, get_node_from_bb_lineno_fuzz
from sra.estimator import Graph, SimpleGraph
from sra.estimator_chunk import structure_estimation
from typing import List

# Get mapping

In [3]:
source_dir = "fuzz-data/ft_data/source_code/totinfo"
source_name = "totinfo_fuzz"

dotfiles = [
    fname
    for fname in os.listdir(source_dir)
    if fname.startswith(".") and fname.endswith(".dot")
]
functions = sorted([fname.split(".")[1] for fname in dotfiles])

bbs_fuzz_path = "fuzz-data/totinfo_exp_1/totinfo_01/totinfo_aflpp_run_01/ft_totinfo.json"
bbs_fuzz = get_bbs_fuzz(bbs_fuzz_path)

debug_info_dict, blocks_dict = parse_ll(
    "fuzz-data/ft_data/source_code/totinfo/totinfo_fuzz.ll"
)
cfgs_intra, node_to_bb = parse_dotfiles(
    dotfiles,
    source_dir,
    debug_info_dict,
    blocks_dict,
    non_overlap_lineidx=False,
    debug=True
)

cfg_inter = gen_cfg_inter(cfgs_intra)
graph = Graph(cfg_inter)

map_fuzz_to_obs_node, map_obs_to_fuzz_node = get_mapping(
    functions, bbs_fuzz, node_to_bb, cfg_inter, debug=True
)


Processing function_name: main
Parsing sanity check len(blocks_dict[function_name])=40 len(nodes)=40, len(edges)=53
Parsing sanity check passed
Processing function_name: InfoTbl
Parsing sanity check len(blocks_dict[function_name])=49 len(nodes)=49, len(edges)=64
Parsing sanity check passed
Processing function_name: QChiSq
Parsing sanity check len(blocks_dict[function_name])=1 len(nodes)=1, len(edges)=0
Parsing sanity check passed
Processing function_name: LGamma
Parsing sanity check len(blocks_dict[function_name])=8 len(nodes)=8, len(edges)=9
Parsing sanity check passed
Processing function_name: QGamma
Parsing sanity check len(blocks_dict[function_name])=4 len(nodes)=4, len(edges)=4
Parsing sanity check passed
Processing function_name: gser
Parsing sanity check len(blocks_dict[function_name])=15 len(nodes)=15, len(edges)=19
Parsing sanity check passed
Processing function_name: gcf
Parsing sanity check len(blocks_dict[function_name])=15 len(nodes)=15, len(edges)=19
Parsing sanity check 

# Load coverage data

In [4]:
from sra.dataprocessor import get_statistics
from tqdm import tqdm

os.makedirs("fuzz-result/totinfo", exist_ok=True)

non_minus_cov_data_o_path = (
    "fuzz-result/totinfo/non_minus_cov_data_o.npy"
)

if os.path.exists(non_minus_cov_data_o_path):
    print(
        """number of trials: 990
covs_f.shape=(356797, 132)
cov_maxs.shape=(356797,)
mean: 155.73216142512408, std: 149.25600284194337, min: 1, max: 1005
quantiles:
          0
0.01   20.0
0.05   32.0
0.10   39.0
0.15   46.0
0.20   51.0
0.25   57.0
0.30   62.0
0.35   66.0
0.40   75.0
0.45   83.0
0.50   93.0
0.55  104.0
0.60  116.0
0.65  136.0
0.70  155.0
0.75  190.0
0.80  246.0
0.85  322.0
0.90  397.0
0.95  524.0
0.99  590.0"""
    )
else:
    projname = "totinfo"
    runs_path = "fuzz-data/totinfo_exp_1"
    run_paths = get_run_paths(runs_path, projname)
    covs_trials_f = []
    for run_path in tqdm(run_paths, desc="Processing runs"):
        for trial_path in get_trial_paths(run_path):
            covs_trial = get_covs_trial(trial_path)
            covs = [cov for cov in covs_trial if max(cov) > 0]
            covs_trials_f.append(covs)
    print(f"number of trials: {len(covs_trials_f)}")
    covs_f = []
    for covs in covs_trials_f:
        covs_f.extend(covs)
    covs_f = np.array(covs_f)
    print(f"{covs_f.shape=}")
    cov_maxs = np.max(covs_f, axis=1)
    print(f"{cov_maxs.shape=}")
    get_statistics(cov_maxs)


number of trials: 990
covs_f.shape=(356797, 132)
cov_maxs.shape=(356797,)
mean: 155.73216142512408, std: 149.25600284194337, min: 1, max: 1005
quantiles:
          0
0.01   20.0
0.05   32.0
0.10   39.0
0.15   46.0
0.20   51.0
0.25   57.0
0.30   62.0
0.35   66.0
0.40   75.0
0.45   83.0
0.50   93.0
0.55  104.0
0.60  116.0
0.65  136.0
0.70  155.0
0.75  190.0
0.80  246.0
0.85  322.0
0.90  397.0
0.95  524.0
0.99  590.0


In [5]:
if os.path.exists(non_minus_cov_data_o_path):
    print(
        """cov_data_f.shape=(322126, 132)
sum_data_f.shape=(132,)
min_obs=14377
total_obs=45157638
min prob: 14377/45157638 = 0.00031837360492592635
* Note. np.max(sum_data_f)=45157621
* There's no one element that represents all the observations."""
    )
else:
    in_between_range = [32 <= cov_max_f <= 524 for cov_max_f in cov_maxs]
    cov_data_f = covs_f[in_between_range]
    print(f"{cov_data_f.shape=}")
    sum_data_f = np.sum(cov_data_f, axis=0)
    print(f"{sum_data_f.shape=}")
    min_obs = np.min(sum_data_f[sum_data_f > 0])
    print(f"{min_obs=}")
    num_obss = cov_maxs[in_between_range]
    total_obs = np.sum(num_obss)
    print(f"{total_obs=}")
    print(f"min prob: {min_obs}/{total_obs} = {min_obs/total_obs}")
    print(f"* Note. {np.max(sum_data_f)=}")
    if max(sum_data_f) != total_obs:
        print("* There's no one element that represents all the observations.")
    else:
        print("* There's one element that represents all the observations.")


cov_data_f.shape=(322126, 132)
sum_data_f.shape=(132,)
min_obs=14377
total_obs=45157638
min prob: 14377/45157638 = 0.00031837360492592635
* Note. np.max(sum_data_f)=45157621
* There's no one element that represents all the observations.


In [6]:
if os.path.exists(non_minus_cov_data_o_path):
    print(
        """
cov_data_o.shape=(322126, 180)
sum_data_o.shape=(180,)
\* Let's also check whether the min probability is still the same after the mapping.
\* _min_obs=14377.0 -> (_min_obs == min_obs)=True"""
    )
else:
    nodenames = list(cfg_inter["nodes"].keys())
    cov_data_o = fuzzdat_to_obs(
        cov_data_f, cfg_inter, bbs_fuzz, map_fuzz_to_obs_node, nodenames
    )
    print(f"{cov_data_o.shape=}")
    sum_data_o = np.sum(cov_data_o, axis=0)
    print(f"{sum_data_o.shape=}")
    _min_obs = np.min(sum_data_o[sum_data_o > 0])
    print(
        "* Let's also check whether the min probability is still the same after the mapping."
    )
    print(f"* {_min_obs=} -> {(_min_obs == min_obs)=}")



cov_data_o.shape=(322126, 180)
sum_data_o.shape=(180,)
\* Let's also check whether the min probability is still the same after the mapping.
\* _min_obs=14377.0 -> (_min_obs == min_obs)=True


In [7]:
observable_nodes_path = "fuzz-result/totinfo/observable_nodes"

if os.path.exists(non_minus_cov_data_o_path):
    non_minus_cov_data_o = np.load(non_minus_cov_data_o_path)
    with open(observable_nodes_path) as f:
        observable_nodes = [line.strip() for line in f.readlines()]
    print(
        """len(non_minus_ids_o)=136
non_minus_cov_data_o.shape=(322126, 136)"""
    )
else:
    non_minus_ids_o = [i for i in range(len(sum_data_o)) if sum_data_o[i] > 0]
    print(f"{len(non_minus_ids_o)=}")
    observable_nodes = [nodenames[i] for i in non_minus_ids_o]
    with open(observable_nodes_path, "w") as f:
        for node in observable_nodes:
            f.write(node + "\n")
    non_minus_cov_data_o = cov_data_o[:, non_minus_ids_o]
    print(f"{non_minus_cov_data_o.shape=}")
    np.save(
        "fuzz-result/totinfo/non_minus_cov_data_o.npy",
        non_minus_cov_data_o,
    )


len(non_minus_ids_o)=136
non_minus_cov_data_o.shape=(322126, 136)


## Data re-organizing

### 1. load _f data and _o data

In [8]:
projname = "totinfo"
runs_path = "fuzz-data/totinfo_exp_1"
run_paths = get_run_paths(runs_path, projname)
covs_trials_f = []
for run_path in run_paths:
    for trial_path in get_trial_paths(run_path):
        covs_trial = get_covs_trial(trial_path)
        covs = [cov for cov in covs_trial if max(cov) > 0]
        covs_trials_f.append(covs)
covs_f = []
for covs in covs_trials_f:
    covs_f.extend(covs)
covs_f = np.array(covs_f)
cov_maxs = np.max(covs_f, axis=1)
in_between_range = [32 <= cov_max_f <= 524 for cov_max_f in cov_maxs]
cov_data_f = covs_f[in_between_range]

non_minus_cov_data_o_path = "fuzz-result/totinfo/non_minus_cov_data_o.npy"
non_minus_cov_data_o = np.load(non_minus_cov_data_o_path)
observable_nodes_path = "fuzz-result/totinfo/observable_nodes"
with open(observable_nodes_path) as f:
    observable_nodes = [line.strip() for line in f.readlines()]


### 2. Check consistency
For quite amount of time, choose random element from random i-th row in the _o data and check if it is in the i-th row of _f data.

In [9]:
num_check = 10000
for _ in range(num_check):
    random_row_idx = np.random.randint(0, non_minus_cov_data_o.shape[0])
    random_val_idx = np.random.randint(0, non_minus_cov_data_o.shape[1])
    random_val_o = non_minus_cov_data_o[random_row_idx, random_val_idx]
    random_row_f = cov_data_f[random_row_idx]
    print(f"{random_row_idx=} {random_val_idx=} {random_val_o=}", end="\r")
    if random_val_o not in random_row_f:
        print(f"{random_row_idx=}")
        print(f"{random_val_idx=}")
        print(f"{non_minus_cov_data_o[random_row_idx, random_val_idx]=}")
        print(
            f"{non_minus_cov_data_o[random_row_idx, random_val_idx]} not in cov_data_f[{random_row_idx}]"
        )
        raise Exception("Not consistent!")


### 3. Split and save _o data

In [10]:
split_ids = []
for covs_trial in covs_trials_f:
    covs_trial_maxs = np.max(np.array(covs_trial), axis=1)
    in_between_range_trial = [
        32 <= cov_max_f <= 524 for cov_max_f in covs_trial_maxs
    ]
    split_ids.append(sum(in_between_range_trial))
split_ids = np.cumsum(split_ids)
assert len(split_ids) == len(covs_trials_f)
assert split_ids[-1] == len(non_minus_cov_data_o)


In [11]:
non_minus_cov_trials_o = np.split(non_minus_cov_data_o, split_ids)
save_dir = os.path.join("fuzz-result/totinfo", "non_minus_cov_trials_o")
os.makedirs(save_dir, exist_ok=True)
for i, non_minus_cov_trial_o in enumerate(non_minus_cov_trials_o):
    np.save(
        os.path.join(save_dir, f"non_minus_cov_trial_o_{i}.npy"),
        non_minus_cov_trial_o,
    )


### 4. load saved _o data

In [12]:
import os

load_dir = os.path.join("fuzz-result/totinfo", "non_minus_cov_trials_o")
non_minus_cov_trial_o_paths = [
    os.path.join(load_dir, fname)
    for fname in sorted(
        [
            fname
            for fname in os.listdir(load_dir)
            if fname.startswith("non_minus_cov_trial_o_")
            and fname.endswith(".npy")
        ],
        key=lambda x: int(x.split("_")[-1].split(".")[0]),
    )
]
non_minus_cov_trials_o = []
for non_minus_cov_trial_o_path in non_minus_cov_trial_o_paths:
    non_minus_cov_trials_o.append(np.load(non_minus_cov_trial_o_path))

non_minus_cov_data_o_path = "fuzz-result/totinfo/non_minus_cov_data_o.npy"
non_minus_cov_data_o = np.load(non_minus_cov_data_o_path)

observable_nodes_path = "fuzz-result/totinfo/observable_nodes"
with open(observable_nodes_path) as f:
    observable_nodes = [line.strip() for line in f.readlines()]

## Implement

#### 0. Statistics of all fuzzing trials coverage data

In [13]:
# Get basic statistics about non_minus_cov_trials_o
total_trials = len(non_minus_cov_trials_o)
total_rows = sum(arr.shape[0] for arr in non_minus_cov_trials_o)
cols = non_minus_cov_trials_o[0].shape[1]

# Calculate sizes
sizes = [arr.shape[0] for arr in non_minus_cov_trials_o]
min_size = min(sizes)
max_size = max(sizes)
avg_size = sum(sizes) / len(sizes)

# Calculate some statistics on the data
all_means = [np.mean(arr) if arr.size > 0 else float('nan') for arr in non_minus_cov_trials_o]
all_maxes = [np.max(arr) if arr.size > 0 else float('-inf') for arr in non_minus_cov_trials_o]
overall_mean = np.nanmean(all_means)
overall_max = max([v for v in all_maxes if v != float('-inf')])

print(f"Statistics for non_minus_cov_trials_o:")
print(f"Number of trials: {total_trials}")
print(f"Total rows across all trials: {total_rows}")
print(f"Number of columns (observable nodes): {cols}")
print(f"Rows per trial - min: {min_size}, max: {max_size}, avg: {avg_size:.2f}")
print(f"Average mean value across all trials: {overall_mean:.2f}")
print(f"Maximum value across all trials: {overall_max}")

# Show first 5 trials shape and some statistical values
print("\nFirst 5 trials details:")
for i in range(min(5, total_trials)):
    arr = non_minus_cov_trials_o[i]
    if arr.size > 0:
        mean_val = np.mean(arr)
        max_val = np.max(arr)
        non_zeros = np.count_nonzero(arr)
    else:
        mean_val = float('nan')
        max_val = float('-inf')
        non_zeros = 0
    print(f"Trial {i}: shape={arr.shape}, mean={mean_val:.2f}, max={max_val}, non-zeros={non_zeros}")


Statistics for non_minus_cov_trials_o:
Number of trials: 991
Total rows across all trials: 322126
Number of columns (observable nodes): 136
Rows per trial - min: 0, max: 361, avg: 325.05
Average mean value across all trials: 20.84
Maximum value across all trials: 524.0

First 5 trials details:
Trial 0: shape=(295, 136), mean=7.98, max=112.0, non-zeros=10125
Trial 1: shape=(296, 136), mean=8.00, max=111.0, non-zeros=10286
Trial 2: shape=(355, 136), mean=7.80, max=112.0, non-zeros=10855
Trial 3: shape=(294, 136), mean=8.01, max=112.0, non-zeros=9888
Trial 4: shape=(298, 136), mean=8.11, max=88.0, non-zeros=10120


### 1. Re-create the distribution with several options

#### This partition the fuzzing trials into two parts:
- start4_coc_data_o_reach:
    - trials where target node was observed at `k-th` time segment, we put trial[:k+1] into it. Note this includes the 1st observation.
- start4_remaining_reach: 
    - trials where target node was never observed
    - trials where target node was observed at the 1st time segment (it is too easy to reach, so we don't want to use it for estimation)
    - trials where target node was observed at `k-th` time segment, we put trial[k+1:] into it

We want to use start4_coc_data_o_reach as the coverage data given to the estimator

In [14]:
def get_start4_cov_data_o(
    target_node, non_minus_cov_trials_o, observable_nodes
):
    cov_data_o = []
    remaining = []
    target_idx = observable_nodes.index(target_node)
    for non_minus_cov_trial_o in non_minus_cov_trials_o:
        # if the target node IS NEVER observed at any time segment in this trial
        # put the trial into REMAINING
        if not non_minus_cov_trial_o[:, target_idx].any():
            remaining.append(non_minus_cov_trial_o)
        # if the target node IS observed at the !1st! time segment in this trial
        # aka right after you start fuzzing
        # put the entire trial into REMAINING
        elif non_minus_cov_trial_o[:, target_idx][0]:
            remaining.append(non_minus_cov_trial_o)
        else:
            # if the target node was observed at some time segment in this trial
            # find the first time segment where the target node was observed
            min_nonzero_idx = np.min(
                np.nonzero(non_minus_cov_trial_o[:, target_idx])[0]
            )
            # partition this trial into two parts:
            # 1. all time segments before and include the first observation
            # 2. everything after 1st observation (note this CAN still contain the target node, 
            # because the time segments are NOT cumulative)
            cov_data_o.append(non_minus_cov_trial_o[: min_nonzero_idx + 1])
            remaining.append(non_minus_cov_trial_o[min_nonzero_idx + 1 :])
    return np.vstack(cov_data_o), np.vstack(remaining)


#### This parition the fuzzing trials into two parts:
- reached_data:
    - trials where target node was observed at some time segment
- not_reached_data:
    - trials where target node was never observed

In [15]:
def partition_coverage_data(
    target_node, non_minus_cov_trials_o, observable_nodes
) -> tuple[np.ndarray, np.ndarray]:
    """
    Partition the coverage data based on if the target node is reached.
    """
    reached_data = []
    not_reached_data = []
    target_idx = observable_nodes.index(target_node)

    for non_minus_cov_trial_o in non_minus_cov_trials_o:
        
        if not non_minus_cov_trial_o[:, target_idx].any():
            not_reached_data.append(non_minus_cov_trial_o)
        else:
            reached_data.append(non_minus_cov_trial_o)
    
    return np.vstack(reached_data), np.vstack(not_reached_data)

### Map the line number in fuzzer coverage data to the node in the graph

In [16]:
# file_name = "totinfo_fuzz.c"
func_name = "InfoTbl"
line_no = 325
target_node, node_info = get_node_from_bb_lineno_fuzz(cfg_inter=cfg_inter, func_name=func_name, line_no=line_no)
print(target_node)
print(node_info)
print("target node is observed:", target_node in observable_nodes)


Node0x61dc66609e60
{'linenums': {325}, 'call': None, 'branch': True, 'program_exit': False, 'nopred': False, 'branch_templete': None, 'func': 'InfoTbl'}
target node is observed: True


#### Verify that partition_coverage_data was done correctly

In [17]:
reached_data, not_reached_data = partition_coverage_data(
    target_node, non_minus_cov_trials_o, observable_nodes
)
print(f"{reached_data.shape=}, {not_reached_data.shape=}")

# verify the partition is correct
target_idx = observable_nodes.index(target_node)
# in reached_data, for all trials, at least one time segment, the target node is observed
if not np.any(reached_data[:, target_idx] > 0):
    print(f"Error: {target_node} is not observed in any reached_data trial.")
    raise Exception("Target node not observed in reached_data.")

# for all trials in not_reached_data, for all time segments, the target node is not observed
if not_reached_data[:, target_idx].any():
    print(f"Error: {target_node} is observed in a not_reached_data trial.")
    raise Exception("Target node observed in not_reached_data.")

reached_data.shape=(213130, 136), not_reached_data.shape=(108996, 136)


In [18]:
print(f"{non_minus_cov_data_o.shape=}")
start4_coc_data_o_reach, start4_remaining_reach = get_start4_cov_data_o(
    target_node, non_minus_cov_trials_o, observable_nodes
)
print(f"{start4_coc_data_o_reach.shape=}, {start4_remaining_reach.shape=}")


non_minus_cov_data_o.shape=(322126, 136)
start4_coc_data_o_reach.shape=(117291, 136), start4_remaining_reach.shape=(204835, 136)


## Evaluate

### 1. Check GT difference

In [19]:
def get_GT_stats(target_idx, cov_data, remaining=None):
    sum_data = np.sum(cov_data, axis=0)
    total_obs = np.max(sum_data)
    target_obs = sum_data[target_idx]
    target_prob = target_obs / total_obs
    if remaining is not None:
        sum_remaining = np.sum(remaining, axis=0)
        total_remaining = np.max(sum_remaining)
        target_remaining = sum_remaining[target_idx]
        target_remaining_prob = target_remaining / total_remaining
    else:
        total_remaining, target_remaining, target_remaining_prob = 0, 0, 0
    return (
        total_obs,
        target_obs,
        target_prob,
        total_remaining,
        target_remaining,
        target_remaining_prob,
    )

In [20]:
total_GT_stat = get_GT_stats(
    observable_nodes.index(target_node),
    non_minus_cov_data_o,
)
print(
    f"[Total       ] total_obs, target_obs, total_remaining, target_remaining = {total_GT_stat[0], total_GT_stat[1], total_GT_stat[3], total_GT_stat[4]}"
)
strat4_GT_stat_reach = get_GT_stats(
    observable_nodes.index(target_node),
    start4_coc_data_o_reach,
    start4_remaining_reach,
)
print(
    f"[Strat4 REACH] total_obs, target_obs, total_remaining, target_remaining = {strat4_GT_stat_reach[0], strat4_GT_stat_reach[1], strat4_GT_stat_reach[3], strat4_GT_stat_reach[4]}"
)


[Total       ] total_obs, target_obs, total_remaining, target_remaining = (np.float64(45157621.0), np.float64(903451.0), 0, 0)
[Strat4 REACH] total_obs, target_obs, total_remaining, target_remaining = (np.float64(16483116.0), np.float64(7561.0), np.float64(28674505.0), np.float64(895890.0))


In [21]:
# from sra.estimator_cum import structure_estimation
for data, data_string in zip([np.zeros((1, len(observable_nodes))),
        not_reached_data,
        start4_coc_data_o_reach,
        non_minus_cov_data_o
    ], ["Empty Coverage across all Basic Blocks",
        "Coverage of runs where target node is not reached",
        "Coverage of runs where target node is reached once, then cut off",
        "Full Coverage of all runs, this is the ground truth"]):
    print("\n" + "=" * 80)
    print("Estimating reachability of the target node...")
    print("Using data:", data_string)
    esti, dist, temp = structure_estimation(
            data,
            graph,
            target_node,
            observable_nodes,
            2,
            # debug=True,
        )
    print("Estimated Reachability of the target node:", esti) # reachability of the target node
    print("Distance from the closest covered ancestor:", dist)
    # print("Maximum coverage among the immediate covered ancestors:", temp)


Estimating reachability of the target node...
Using data: Empty Coverage across all Basic Blocks
Estimated Reachability of the target node: 0.0009765625
Distance from the closest covered ancestor: 9

Estimating reachability of the target node...
Using data: Coverage of runs where target node is not reached
Estimated Reachability of the target node: 8.949491932100332e-08
Distance from the closest covered ancestor: 2

Estimating reachability of the target node...
Using data: Coverage of runs where target node is reached once, then cut off
Estimated Reachability of the target node: 0.00045871161672457337
Distance from the closest covered ancestor: 0

Estimating reachability of the target node...
Using data: Full Coverage of all runs, this is the ground truth
Estimated Reachability of the target node: 0.020006604419832586
Distance from the closest covered ancestor: 0


In [22]:
from sra.dataprocessor import (
    strat4_save_data,
    total_frac_strategy,
    total_frac_draw_graph,
)


def analyze_node_strat4(
    target_node, cov_data, remain_data, observable_nodes, graph, option=""
):
    esti, dist, _ = structure_estimation(
        np.zeros((1, len(observable_nodes))),
        graph,
        target_node,
        observable_nodes,
        2,
    )
    dirpath = f"fuzz-result/totinfo/start4/{target_node}-{esti:.2e}-{dist}"
    (
        GT,
        lap_esti_df,
        gt_esti_df,
        gt_unob_esti_df,
        struct_esti_df,
    ) = total_frac_strategy(
        target_node, cov_data, observable_nodes, graph, dirpath, prefix=option
    )
    print(f"{GT=}")
    print(f"reciprocal: {1/GT}")

    # save
    GT_stat = get_GT_stats(
        observable_nodes.index(target_node),
        cov_data,
        remain_data,
    )
    strat4_save_data(
        dirpath,
        GT_stat,
        lap_esti_df,
        gt_esti_df,
        gt_unob_esti_df,
        struct_esti_df,
        option,
    )


strat4_coc_data_o = start4_coc_data_o_reach
start4_remaining = start4_remaining_reach
analyze_node_strat4(
    target_node,
    strat4_coc_data_o,
    start4_remaining,
    observable_nodes,
    graph,
)


GT=0.00045871181152883956
partial_idx=0, total_frac_trial_partial.shape=(2180, 136)


/mnt/ssd3/ericzhou/sra-artifact/rq2/sra/dataprocessor.py:26: RuntimeWarning: divide by zero encountered in log10
  return np.abs(np.log10(esti) - np.log10(GT))


child=('Node0x61dc665ee260', <EdgeType.INTRA: 1>, <BranchType.TRUE: 1>)
child=('Node0x61dc665ee2f0', <EdgeType.INTRA: 1>, <BranchType.FALSE: 2>)
For now, let's keep the source node is a uncovered source node, counterfactual_prob = laplace
source_node='Node0x61dc665ebff0', num_covered=1.0, counterfactual_prob=1.0 * 0.4 = 0.4
np.mean(struct_times)=np.float64(0.004140900143789589)
GT=np.float64(0.00045871181152883956)
reciprocal: 2180.0179870387515
